In [ ]:
import array, csv, json, math
from osgeo import ogr
from random import shuffle, uniform
from shapely.geometry import shape, Point, Polygon

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromPolygocn(geom):
    polygon = shape(geom)
    bbox = polygon.bounds
    l,b,r,t = bbox
    while True:
        point = Point(uniform(l,r),uniform(t,b))
        if point is None:
            break
        if polygon.contains(point):
            break
    return point.__geo_interface__['coordinates']

In [ ]:
# Transform NI Small Areas shapefile into GeoJSON
cmd = "ogr2ogr -f GeoJSON -t_srs crs:84 SA2011/SA2011.geojson SA2011/SA2011.shp"
!$cmd

In [ ]:
# Load small areas GeoJSON file
with open("SA2011/SA2011.geojson") as f:
    sa2011 = json.load(f)

In [ ]:
#sa2011['features'][0]['properties']
#{u'Hectares': 169.044,
# u'SA2011': u'N00000002',
# u'SOA2011': u'95AA01S2',
# u'X_COORD': 315465.0,
# u'Y_COORD': 377176.0}

In [ ]:
belfastData = []
with open("Geographic Data (statistical geographies).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['HSCT'] == "Belfast HSCT":
            belfastData.append(row)

In [ ]:
#belfastData[0]
#{'AA1998': 'BELFAST WEST',
# 'AA2008': 'BELFAST WEST',
# 'HSCT': 'Belfast HSCT',
# 'LGD1992NAME': 'BELFAST',
# 'SA': 'N00000825',
# 'SOA': '95GG01S1',
# 'SQ Km': '0.11',
# 'Settlement2015(1)': 'BELFAST CITY',
# 'Settlement2015(2)': '',
# 'Settlement2015(3)': '',
# 'URBAN_RURAL(2015)': 'Urban',
# 'WARD1992': 'ANDERSONSTOWN'}

In [ ]:
belfastGeoJSON = { 
    "type": "FeatureCollection",
    "features": []
}
for row in belfastData:
    sa = row['SA']
    for feature in sa2011['features']:
        if feature['properties']['SA2011'] == sa:
            for key in row.keys():
                feature['properties'][key] = row[key]
            belfastGeoJSON['features'].append(feature)
            break
with open('belfast.geojson', 'w') as outfile:  
    json.dump(belfastGeoJSON, outfile, indent=4)

In [ ]:
with open('belfast.geojson') as f:  
    belfastGeoJSON = json.load(f)

In [ ]:
belfastSa2Idx = {}
for i in range(0,len(belfastGeoJSON["features"])):
    feature = belfastGeoJSON["features"][i]
    sa = feature['properties']['SA']
    belfastSa2Idx[sa] = i
len(belfastSa2Idx)

In [ ]:
religionData_2011 = []
with open("_KS211NI (s).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA Code'] in belfastSa2Idx:
            religionData_2011.append(row)
len(religionData_2011)

In [ ]:
#religionData_2011[0]
#{'All usual residents': '367',
# 'Religion: Catholic': '320',
# 'Religion: Catholic (%)': '87.19',
# 'Religion: Church of Ireland': '0',
# 'Religion: Church of Ireland (%)': '0.00',
# 'Religion: Methodist Church in Ireland': '0',
# 'Religion: Methodist Church in Ireland (%)': '0.00',
# 'Religion: No religion': '15',
# 'Religion: No religion (%)': '4.09',
# 'Religion: Other Christian (including Christian related)': '3',
# 'Religion: Other Christian (including Christian related) (%)': '0.82',
# 'Religion: Other religions': '0',
# 'Religion: Other religions (%)': '0.00',
# 'Religion: Presbyterian Church in Ireland': '2',
# 'Religion: Presbyterian Church in Ireland (%)': '0.54',
# 'Religion: Religion not stated': '27',
# 'Religion: Religion not stated (%)': '7.36',
# 'SA': 'N00000825 (Andersonstown ward)',
# 'SA Code': 'N00000825'}

In [128]:
romanCatholic = ['Religion: Catholic']
presbyterian = ['Religion: Presbyterian Church in Ireland']
churchOfIreland = ['Religion: Church of Ireland']
methodist = ['Religion: Methodist Church in Ireland']
other = ['Religion: No religion', 'Religion: Other Christian (including Christian related)', 
         'Religion: Other religions', 'Religion: Religion not stated']

religions = {
    'romanCatholic':romanCatholic, 
    'presbyterian': presbyterian, 
    'churchOfIreland': churchOfIreland, 
    'methodist': methodist, 
    'other': other}

religion2Idx = {
    'romanCatholic': 1.0, 
    'presbyterian': 2.0, 
    'churchOfIreland': 3.0, 
    'methodist': 4.0, 
    'other': 5.0
}

In [ ]:
geom = belfastGeoJSON["features"][0]["geometry"]
p = RandomPointFromPolygon(geom)
LonLatToPixelXY(p)

In [ ]:
points_2011 = []
for row in religionData_2011:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2011.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [ ]:
len(points_2011)

In [ ]:
shuffle(points_2011)

In [ ]:
points = []
for sublist in points_2011:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2011.bin', 'wb'))

In [ ]:
lookupTableData = []
with open("Look-up Tables_0.xlsx - SA2011.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA2011'] in belfastSa2Idx:
            lookupTableData.append(row)
len(lookupTableData)    

In [ ]:
#lookupTableData[0]
#{'COA2001_1': '95GG010001',
#'COA2001_2': '',
# 'COA2001_3': '',
# 'COA2001_4': '',
# 'SA2011': 'N00000825',
# 'SA2011NAME': 'N00000825 (Andersonstown ward)',
# 'SOA2001': '95GG01S1',
# 'SOANAME': 'Andersonstown_1'}

In [ ]:
raw_2001 = []
with open("2001/oa/TABLE CAS308 (OA).csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        raw_2001.append(row)


In [ ]:
religionDataIdx_2001 = {}

for row in lookupTableData:
    idx_2001 = row['COA2001_1']
    religionDataIdx_2001[idx_2001] = []
    for raw_row in raw_2001:
        if len(raw_row) == 3:
            raw_idx = raw_row[0].strip()
            raw_key = raw_row[1]
            raw_value = raw_row[2]
            if raw_value == "-":
                raw_value = '0'
            if raw_idx == idx_2001:
                religionDataIdx_2001[idx_2001].append({raw_key: raw_value})
            

In [ ]:
religionData_2001 = []
for row in lookupTableData:
    data_2001 = {}
    idx_2001 = row['COA2001_1']
    data_2001['SA Code'] = row['SA2011']
    data_2001['COA2001_1'] = idx_2001
    data_2001['All usual residents'] = religionDataIdx_2001[idx_2001][0]['All persons']
    data_2001['Religion: Catholic'] = religionDataIdx_2001[idx_2001][2]['Catholic']
    data_2001['Religion: Church of Ireland'] = religionDataIdx_2001[idx_2001][4]['Church of Ireland']
    data_2001['Religion: Methodist Church in Ireland'] = religionDataIdx_2001[idx_2001][5]['Methodist Church in Ireland']
    data_2001['Religion: Presbyterian Church in Ireland'] = religionDataIdx_2001[idx_2001][3]['Presbyterian Church in Ireland']
    data_2001['Religion: Other Christian (including Christian related)'] = religionDataIdx_2001[idx_2001][6]['Other Christian (including Christian related)']
    data_2001['Religion: Other religions'] = religionDataIdx_2001[idx_2001][7]['Other religions and philosophies']
    data_2001['Religion: No religion'] = religionDataIdx_2001[idx_2001][8]['No religion or religion not stated']
    data_2001['Religion: Religion not stated'] = '0'
    religionData_2001.append(data_2001)
    
    

In [ ]:
religionData_2001[0]

In [ ]:
religionDataIdx_2001['95GG010002']

In [ ]:
data_2001

In [ ]:
religionDataIdx_2001[idx_2001]

In [ ]:
points_2001 = []
for row in religionData_2001:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2001.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [ ]:
len(points_2001)

In [ ]:
shuffle(points_2001)

In [ ]:
points = []
for sublist in points_2001:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2001.bin', 'wb'))

In [ ]:
grid2OA = []
with open("5672/5672package/package/100mgrids/txt/grid100mOA.txt") as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        grid2OA.append(row)

In [ ]:
#grid2OA[0]
#{'Grid01': 'J 075 899',
# 'Grid718191': 'J075899 - J076900',
# 'OA': '95AA050006',
# 'Oa_code': '6',
# 'Oa_cty': '95',
# 'Oa_dis': 'AA',
# 'Oa_ward': '5',
# 'Ward': '95AA05',
# 'X': '307550',
# 'Y': '389950'}

In [ ]:
oa2Grid718191 = {}
for row in grid2OA:
    if not row["OA"] in oa2Grid718191:
        oa2Grid718191[row["OA"]] = []
    oa2Grid718191[row["OA"]].append(row['Grid718191'])

In [ ]:
#oa2Grid718191["95GG400001"]
#['J321744 - J322745',
# 'J321745 - J322746',
# 'J321746 - J322747',
# 'J322745 - J323746',
# 'J322747 - J323748']

In [ ]:
# Convert DBF to CSV
years = ["1971", "1981", "1991"]
for year in years:
    cmd = "ogr2ogr -f CSV %soth100m.csv 5672/5672package/package/100mgrids/dbf/%soth100m.dbf" % (year, year)
    !$cmd
   

In [ ]:
data_1991 = []
with open("1991oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1991.append(row)
    

In [ ]:
# data_1991[0]
# {'GRID': 'C414160 - C415161',
#   ...
# 'V783_91': '-99997',
# 'V784_91': '-99997',
# 'V785_91': '-99997',
# 'V786_91': '-99997',
# 'V787_91': '-99997',
# 'V788_91': '-99997',
# 'V789_91': '-99997',
# 'V790_91': '-99997',
# 'V791_91': '-99997',
# 'V792_91': '-99997',
# 'V793_91': '-99997',
# 'V794_91': '-99997',
# ...
# }

In [ ]:
# 1971 and 1981
#     totalCatholic = int(row['V783_91']) + int(row['V784_91'])
#    totalPresbyterian = int(row['V785_91']) + int(row['V786_91'])
#    totalCoI = int(row['V787_91']) + int(row['V788_91'])
#    totalMethodist = int(row['V789_91']) + int(row['V790_91'])
#    totalOther = int(row['V791_91']) + int(row['V792_91'])    
#    totalNotStated = int(row['V793_91']) + int(row['V794_91'])

grid2religion_1991 = {}
for row in data_1991:
    gridIdx = row['GRID']
    totalCatholic = int(row['V895_91']) + int(row['V896_91'])
    totalPresbyterian = int(row['V897_91']) + int(row['V898_91'])
    totalCoI = int(row['V899_91']) + int(row['V900_91'])
    totalMethodist = int(row['V901_91']) + int(row['V902_91'])
    totalOther = int(row['V903_91']) + int(row['V904_91'])    
    totalNone = int(row['V905_91']) + int(row['V906_91'])
    totalNotStated = int(row['V907_91']) + int(row['V908_91'])
    grid2religion_1991[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated),
        'None': max(0,totalNone)
    }
    

In [ ]:
religionData_1991 = []
for row in lookupTableData:
    data_1991 = {}
    idx_2001 = row['COA2001_1']
    data_1991['SA Code'] = row['SA2011']
    data_1991['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1991:
                totalCatholic += grid2religion_1991[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1991[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1991[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1991[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1991[gridIdx]['Other']
                totalNotStated += grid2religion_1991[gridIdx]['Religion not stated']
                totalNone += grid2religion_1991[gridIdx]['None']
        
    data_1991['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated + totalNone
    data_1991['Religion: Catholic'] = totalCatholic
    data_1991['Religion: Church of Ireland'] = totalCoI
    data_1991['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1991['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1991['Religion: Other Christian (including Christian related)'] = totalOther
    data_1991['Religion: Other religions'] = 0
    data_1991['Religion: No religion'] = totalNone
    data_1991['Religion: Religion not stated'] = totalNotStated
    religionData_1991.append(data_1991)


In [ ]:
religionData_1991[1000]

In [ ]:
found = 0
missed = 0
for row in grid2OA:
    gridIdx = row['Grid718191']
    matched = False
    for val in data_1991:        
        if gridIdx == val['GRID']:
            matched = True
            break
    if matched:
        found += 1
    else:
        missed += 1 

In [ ]:
missed

In [ ]:
found

In [ ]:
points_1991 = []
for row in religionData_1991:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1991.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

In [ ]:
len(points_1991)

In [ ]:
shuffle(points_1991)
points = []
for sublist in points_1991:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1991.bin', 'wb'))

In [ ]:
data_1981 = []
with open("1981oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1981.append(row)
    

# 1971 and 1981

grid2religion_1981 = {}
for row in data_1981:
    gridIdx = row['GRID']
    totalCatholic = int(row['V783_81']) + int(row['V784_81'])
    totalPresbyterian = int(row['V785_81']) + int(row['V786_81'])
    totalCoI = int(row['V787_81']) + int(row['V788_81'])
    totalMethodist = int(row['V789_81']) + int(row['V790_81'])
    totalOther = int(row['V781_81']) + int(row['V792_81'])    
    totalNotStated = int(row['V793_81']) + int(row['V794_81'])
    grid2religion_1981[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated)
    }
    

religionData_1981 = []
for row in lookupTableData:
    data_1981 = {}
    idx_2001 = row['COA2001_1']
    data_1981['SA Code'] = row['SA2011']
    data_1981['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1981:
                totalCatholic += grid2religion_1981[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1981[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1981[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1981[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1981[gridIdx]['Other']
                totalNotStated += grid2religion_1981[gridIdx]['Religion not stated']
                
    data_1981['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated 
    data_1981['Religion: Catholic'] = totalCatholic
    data_1981['Religion: Church of Ireland'] = totalCoI
    data_1981['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1981['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1981['Religion: Other Christian (including Christian related)'] = totalOther
    data_1981['Religion: Other religions'] = 0
    data_1981['Religion: No religion'] = totalNone
    data_1981['Religion: Religion not stated'] = totalNotStated
    religionData_1981.append(data_1981)



In [ ]:
points_1981 = []
for row in religionData_1981:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1981.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

shuffle(points_1981)
points = []
for sublist in points_1981:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1981.bin', 'wb'))


In [ ]:
len(points_1981)

In [ ]:
data_1971 = []
with open("1971oth100m.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data_1971.append(row)
    

# 1971 and 1971

grid2religion_1971 = {}
for row in data_1971:
    gridIdx = row['GRID']
    totalCatholic = int(row['V783_71']) + int(row['V784_71'])
    totalPresbyterian = int(row['V785_71']) + int(row['V786_71'])
    totalCoI = int(row['V787_71']) + int(row['V788_71'])
    totalMethodist = int(row['V789_71']) + int(row['V790_71'])
    totalOther = int(row['V771_71']) + int(row['V792_71'])    
    totalNotStated = int(row['V793_71']) + int(row['V794_71'])
    grid2religion_1971[gridIdx] = {
        'Catholic': max(0,totalCatholic),
        'Church of Ireland': max(0,totalCoI),
        'Methodist Church in Ireland': max(0,totalMethodist),
        'Presbyterian Church in Ireland': max(0,totalPresbyterian),
        'Other': max(0,totalOther),
        'Religion not stated': max(0,totalNotStated)
    }


religionData_1971 = []
for row in lookupTableData:
    data_1971 = {}
    idx_2001 = row['COA2001_1']
    data_1971['SA Code'] = row['SA2011']
    data_1971['COA2001_1'] = idx_2001
    totalCatholic = 0
    totalPresbyterian = 0
    totalCoI = 0
    totalMethodist = 0
    totalOther = 0 
    totalNotStated = 0
    totalNone = 0
    if idx_2001 in oa2Grid718191:
        for gridIdx in oa2Grid718191[idx_2001]:        
            if gridIdx in grid2religion_1971:
                totalCatholic += grid2religion_1971[gridIdx]['Catholic']
                totalPresbyterian += grid2religion_1971[gridIdx]['Presbyterian Church in Ireland']
                totalCoI += grid2religion_1971[gridIdx]['Church of Ireland']
                totalMethodist += grid2religion_1971[gridIdx]['Methodist Church in Ireland']
                totalOther += grid2religion_1971[gridIdx]['Other']
                totalNotStated += grid2religion_1971[gridIdx]['Religion not stated']
                
    data_1971['All usual residents'] = totalCatholic + totalPresbyterian + totalCoI + totalMethodist + totalOther + totalNotStated 
    data_1971['Religion: Catholic'] = totalCatholic
    data_1971['Religion: Church of Ireland'] = totalCoI
    data_1971['Religion: Methodist Church in Ireland'] = totalMethodist
    data_1971['Religion: Presbyterian Church in Ireland'] = totalPresbyterian
    data_1971['Religion: Other Christian (including Christian related)'] = totalOther
    data_1971['Religion: Other religions'] = 0
    data_1971['Religion: No religion'] = totalNone
    data_1971['Religion: Religion not stated'] = totalNotStated
    religionData_1971.append(data_1971)



In [ ]:
points_1971 = []
for row in religionData_1971:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = row['All usual residents']
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += row[i]
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_1971.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
print mis        

shuffle(points_1971)
points = []
for sublist in points_1971:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-1971.bin', 'wb'))



In [ ]:
len(points_1971)

In [236]:
censusTotalsOverTime = {}
for row in religionData_1971:
    idx = row["SA Code"]
    if not idx in censusTotalsOverTime:
        censusTotalsOverTime[idx] = {}
    censusTotalsOverTime[idx]['1971'] = {
        'romanCatholic': int(row['Religion: Catholic']),
        'churchOfIreland': int(row['Religion: Church of Ireland']),
        'methodist': int(row['Religion: Methodist Church in Ireland']),
        'presbyterian': int(row['Religion: Presbyterian Church in Ireland']),
        'other': int(row['Religion: No religion'])  + int(row['Religion: Other Christian (including Christian related)']) + int(row['Religion: Other religions']) + int(row['Religion: Religion not stated'])
    }
    
for row in religionData_1981:
    idx = row["SA Code"]
    if not idx in censusTotalsOverTime:
        censusTotalsOverTime[idx] = {}
    censusTotalsOverTime[idx]['1981'] = {
        'romanCatholic': int(row['Religion: Catholic']),
        'churchOfIreland': int(row['Religion: Church of Ireland']),
        'methodist': int(row['Religion: Methodist Church in Ireland']),
        'presbyterian': int(row['Religion: Presbyterian Church in Ireland']),
        'other': int(row['Religion: No religion'])  + int(row['Religion: Other Christian (including Christian related)']) + int(row['Religion: Other religions']) + int(row['Religion: Religion not stated'])
    }
    
for row in religionData_1991:
    idx = row["SA Code"]
    if not idx in censusTotalsOverTime:
        censusTotalsOverTime[idx] = {}
    censusTotalsOverTime[idx]['1991'] = {
        'romanCatholic': int(row['Religion: Catholic']),
        'churchOfIreland': int(row['Religion: Church of Ireland']),
        'methodist': int(row['Religion: Methodist Church in Ireland']),
        'presbyterian': int(row['Religion: Presbyterian Church in Ireland']),
        'other': int(row['Religion: No religion'])  + int(row['Religion: Other Christian (including Christian related)']) + int(row['Religion: Other religions']) + int(row['Religion: Religion not stated'])
    }
    
for row in religionData_2001:
    idx = row["SA Code"]
    if not idx in censusTotalsOverTime:
        censusTotalsOverTime[idx] = {}
    censusTotalsOverTime[idx]['2001'] = {
        'romanCatholic': int(row['Religion: Catholic']),
        'churchOfIreland': int(row['Religion: Church of Ireland']),
        'methodist': int(row['Religion: Methodist Church in Ireland']),
        'presbyterian': int(row['Religion: Presbyterian Church in Ireland']),
        'other': int(row['Religion: No religion'])  + int(row['Religion: Other Christian (including Christian related)']) + int(row['Religion: Other religions']) + int(row['Religion: Religion not stated'])
    }
    
for row in religionData_2011:
    idx = row["SA Code"]
    if not idx in censusTotalsOverTime:
        censusTotalsOverTime[idx] = {}
    censusTotalsOverTime[idx]['2011'] = {
        'romanCatholic': int(row['Religion: Catholic'].replace(",","")),
        'churchOfIreland': int(row['Religion: Church of Ireland']),
        'methodist': int(row['Religion: Methodist Church in Ireland']),
        'presbyterian': int(row['Religion: Presbyterian Church in Ireland']),
        'other': int(row['Religion: No religion'])  + int(row['Religion: Other Christian (including Christian related)']) + int(row['Religion: Other religions']) + int(row['Religion: Religion not stated'])
    }

In [124]:
len(range(1971,2012))

41

In [237]:
religionData_1971[0]

{'All usual residents': 511,
 'COA2001_1': '95GG010001',
 'Religion: Catholic': 339,
 'Religion: Church of Ireland': 3,
 'Religion: Methodist Church in Ireland': 0,
 'Religion: No religion': 0,
 'Religion: Other Christian (including Christian related)': 0,
 'Religion: Other religions': 0,
 'Religion: Presbyterian Church in Ireland': 1,
 'Religion: Religion not stated': 168,
 'SA Code': 'N00000825'}

In [238]:
censusTotalsOverTime[idx]

{'1971': {'churchOfIreland': 84,
  'methodist': 70,
  'other': 34,
  'presbyterian': 174,
  'romanCatholic': 115},
 '1981': {'churchOfIreland': 40,
  'methodist': 46,
  'other': 96,
  'presbyterian': 119,
  'romanCatholic': 114},
 '1991': {'churchOfIreland': 31,
  'methodist': 20,
  'other': 79,
  'presbyterian': 77,
  'romanCatholic': 153},
 '2001': {'churchOfIreland': 27,
  'methodist': 26,
  'other': 68,
  'presbyterian': 43,
  'romanCatholic': 130},
 '2011': {'churchOfIreland': 26,
  'methodist': 16,
  'other': 77,
  'presbyterian': 30,
  'romanCatholic': 135}}

In [265]:
totalsPerYear = {}
years = ['1971', '1981', '1991', '2001', '2011']
religions = ["romanCatholic", "churchOfIreland", "methodist", "other", "presbyterian"]
points = {}
count = 0
for row in religionData_2011:
    idx = row["SA Code"]
    count += 1
    if count % 100 == 0:
        print count
        
    if not idx in totalsPerYear:
        totalsPerYear[idx] = {}

    if not idx in points:
        points[idx] = []

    for year in range(len(years)-1):
        nextYear = years[year+1]
        currYear = years[year]
        for religion in religions:        
            currentValue = int(censusTotalsOverTime[idx][currYear][religion])
            nextValue = int(censusTotalsOverTime[idx][nextYear][religion])
            diff = nextValue  - currentValue
            step = math.floor(diff/10)
            if not key in totalsPerYear[idx]:
                totalsPerYear[idx][religion] = []
            for i in range(10):
                if step < 0:
                    value = max(i*step + currentValue, nextValue)
                else:
                    value = min(i*step + currentValue, nextValue)               
                totalsPerYear[idx][religion].append(value)
            if (nextYear == '2011'):
                totalsPerYear[idx][religion].append(nextValue)

    geom = belfastGeoJSON["features"][belfastSa2Idx[idx]]['geometry']
    for religion in religions:
        value = religion2Idx[religion]
        year_0 = 1971

        for i in range(int(totalsPerYear[idx][religion][0])):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(value)
            point.append(year_0)
            points[idx].append(point)

        for i in range(1,len(totalsPerYear[idx][religion])):
            currentYear = i + 1971
            lastYear = i-1+1971
            currentYearValue = int(totalsPerYear[idx][religion][i])
            lastYearValue = int(totalsPerYear[idx][religion][i-1])
            difference = currentYearValue - lastYearValue
            if difference < 0:        
                absDiff = abs(difference)
                for j in range(absDiff):
                    for k in range(len(points[idx])):
                        if len(points[idx][k]) == 4:
                            points[idx][k].append(currentYear)
                            break
            if difference > 0:
                for j in range(difference):
                    point = []
                    point += LonLatToPixelXY(RandomPointFromPolygon(geom))
                    point.append(value)
                    point.append(currentYear)
                    points[idx].append(point)                
                
        for point in points[idx]:
            if len(point) == 4:
                point.append(2012)

100
200


KeyboardInterrupt: 

In [266]:
len(points)

168936

In [259]:
len(religionData_2011)

1008

In [264]:
200 % 100

0

In [267]:
pointas = []

In [275]:
totalsPerYear = {}
years = ['1971', '1981', '1991', '2001', '2011']
religions = ["romanCatholic", "churchOfIreland", "methodist", "other", "presbyterian"]
points = {}
count = 0
for row in religionData_2011:
    idx = row["SA Code"]
    count += 1
    if count % 100 == 0:
        print count
        
    if not idx in totalsPerYear:
        totalsPerYear[idx] = {}

    if not idx in points:
        points[idx] = {}
        for religion in religions:
            points[idx][religion] = []

    for year in range(len(years)-1):
        nextYear = years[year+1]
        currYear = years[year]
        for religion in religions:        
            currentValue = int(censusTotalsOverTime[idx][currYear][religion])
            nextValue = int(censusTotalsOverTime[idx][nextYear][religion])
            diff = nextValue  - currentValue
            step = math.floor(diff/10)
            if not key in totalsPerYear[idx]:
                totalsPerYear[idx][religion] = []
            for i in range(10):
                if step < 0:
                    value = max(i*step + currentValue, nextValue)
                else:
                    value = min(i*step + currentValue, nextValue)               
                totalsPerYear[idx][religion].append(value)
            if (nextYear == '2011'):
                totalsPerYear[idx][religion].append(nextValue)


100
200
300
400
500
600
700
800
900
1000


In [276]:
count = 0
for row in religionData_2011:
    idx = row["SA Code"]
    count += 1
    if count % 100 == 0:
        print count

    geom = belfastGeoJSON["features"][belfastSa2Idx[idx]]['geometry']
    for religion in religions:
        value = religion2Idx[religion]
        year_0 = 1971

        for i in range(int(totalsPerYear[idx][religion][0])):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(value)
            point.append(year_0)
            points[idx][religion].append(point)

        for i in range(1,len(totalsPerYear[idx][religion])):
            currentYear = i + 1971
            lastYear = i-1+1971
            currentYearValue = int(totalsPerYear[idx][religion][i])
            lastYearValue = int(totalsPerYear[idx][religion][i-1])
            difference = currentYearValue - lastYearValue
            if difference < 0:        
                absDiff = abs(difference)
                for j in range(absDiff):
                    for k in range(len(points[idx][religion])):
                        if len(points[idx][religion][k]) == 4:
                            points[idx][religion][k].append(currentYear)
                            break
            if difference > 0:
                for j in range(difference):
                    point = []
                    point += LonLatToPixelXY(RandomPointFromPolygon(geom))
                    point.append(value)
                    point.append(currentYear)
                    points[idx][religion].append(point)                
                
        for point in points[idx][religion]:
            if len(point) == 4:
                point.append(2012)

100
200
300
400
500
600
700
800
900
1000


In [278]:
points[idx][religion]

[[123.80295572168285, 81.5022631519091, 2.0, 1971, 1972],
 [123.80270029522805, 81.50340568767945, 2.0, 1971, 1972],
 [123.80384906635645, 81.5023770495392, 2.0, 1971, 1972],
 [123.80439588259671, 81.50348366519913, 2.0, 1971, 1972],
 [123.80279085600883, 81.50295758153962, 2.0, 1971, 1972],
 [123.80348264418583, 81.50271577007626, 2.0, 1971, 1972],
 [123.80292508908165, 81.50199193377506, 2.0, 1971, 1973],
 [123.80318413555435, 81.50224030771682, 2.0, 1971, 1973],
 [123.80319978071279, 81.50319638468699, 2.0, 1971, 1973],
 [123.8029354082663, 81.50203102522313, 2.0, 1971, 1973],
 [123.80491626072101, 81.50327287011952, 2.0, 1971, 1973],
 [123.80280068215309, 81.50193379244487, 2.0, 1971, 1973],
 [123.80295156925564, 81.50369875417783, 2.0, 1971, 1974],
 [123.80291606276573, 81.50366953373792, 2.0, 1971, 1974],
 [123.80366978201847, 81.50153814051234, 2.0, 1971, 1974],
 [123.80240012907078, 81.50211969384847, 2.0, 1971, 1974],
 [123.80301881058, 81.50309279877482, 2.0, 1971, 1974],
 [1

In [283]:
sortedPoints = []
for row in religionData_2011:
    idx = row["SA Code"]
    count += 1
    if count % 100 == 0:
        print count

    for religion in religions:
        sortedPoints += points[idx][religion]

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [284]:
len(sortedPoints)

626705

In [285]:
shuffle(sortedPoints)
animatedPoints = []
for sublist in sortedPoints:
    for item in sublist:
        animatedPoints.append(item)
array.array('f', animatedPoints).tofile(open('animated-dotmap.bin', 'wb'))




In [286]:
len(animatedPoints)

3133525

In [287]:
animatedPoints[0:10]

[123.75895712990682,
 81.46188656014272,
 1.0,
 1971,
 1981,
 123.75116176914402,
 81.52266575585529,
 5.0,
 1972,
 2012]

In [292]:
count_1971 = 0
for point in sortedPoints:
    if point[3] == 1971:
        count_1971 += 1

In [293]:
count_1971

362403

In [291]:
point

[123.77130595661673, 81.46950354030605, 5.0, 2001, 2012]